In [2]:
import torch
import matplotlib.pyplot as plt
import numpy as np
from monai.transforms import LoadImaged, MapTransform, Compose, ToNumpyd
from monai.data import Dataset


In [26]:
import glob
Validation_Cases = list(glob.glob("./MICCAI_BraTS2020_TrainingData/BraTS20*"))

In [28]:
experiment_cases = []

for case in Validation_Cases:
    try:
        t1 = list(glob.glob(f"{case}/*t1.nii.gz"))
        t1ce = list(glob.glob(f"{case}/*t1ce.nii.gz"))
        t2 = list(glob.glob(f"{case}/*t2.nii.gz"))
        flair = list(glob.glob(f"{case}/*flair.nii.gz"))
        seg = glob.glob(f"{case}/*seg.nii.gz")[0]

        img = t1 + t1ce + t2 + flair

        d = {"image": img, "seg": seg}
        experiment_cases.append(d)
    except Exception as e:
        print(e)
        print(case)


In [30]:
class ConvertLabelsIntoOneHotd(MapTransform):
    """Takes input tensor of segmentation which contains
    values in set (0,1,2,4) where\n
    0 -> Background/Normal\n
    1 -> Non- Enhancing Tumor Core\n
    2 -> Edema\n
    4 -> Enhancing tumor core\n

    and returns a one hot encoded 3 channel tensor where
    1st Channel -> Whole tumor (1,2 and 4)
    2nd Channel -> Tumor Core (1 and 4)
    3rd Channel -> Enhancing Tumor core (4)
    """
    def __call__(self, data):
        data_dict = dict(data)
        for key in self.keys:
            one_hot_encode_array = [
                torch.logical_or(
                    torch.logical_or(data_dict[key] == 1, data_dict[key] == 2),
                    data_dict[key] == 4,
                ), # Whole Tumor
                torch.logical_or(data_dict[key] == 1, data_dict[key] == 4), # Tumor Core
                data_dict[key] == 4, # Enhancing Core
            ]
            data_dict[key] = torch.stack(one_hot_encode_array, axis=0).astype(torch.int32)
        return data_dict

In [31]:
transform_for_analysis = Compose(
    [
        LoadImaged(keys=["image", "seg"]),
        ConvertLabelsIntoOneHotd(keys="seg"),
        ToNumpyd(keys=["image", "seg"])
    ]
)


In [32]:
dataset = Dataset(experiment_cases,transform_for_analysis)

## Textural Analysis

1. Take GLCM from Tumor Boundary

In [33]:
# functions to find boundary from segmentation
from scipy import ndimage

In [34]:

case = 25
wt = dataset[case]['seg'][0]
tc = dataset[case]['seg'][1]
et = dataset[case]['seg'][2]

t1 = dataset[case]['image'][0]
t1ce = dataset[case]['image'][1]
t2 = dataset[case]['image'][2]
flair = dataset[case]['image'][3]


In [35]:
blob_wt = ndimage.find_objects(wt)
print(f"{len(blob_wt)} WT blob found")
blob_tc = ndimage.find_objects(tc)
print(f"{len(blob_tc)} TC blob found")
blob_et = ndimage.find_objects(et)
print(f"{len(blob_et)} ET blob found")

1 WT blob found
1 TC blob found
1 ET blob found


In [37]:
# Finding Center of Mass and Volume

print("For Whole Tumor")
for i,blob in enumerate(blob_wt):
    print(f"Blob {i}: coordinate {ndimage.center_of_mass(wt[blob])} and {np.count_nonzero(wt[blob]) / 10**3} cm^3")


print("\nFor Tumor Core")
for i,blob in enumerate(blob_tc):
    print(f"Blob {i}: coordinate {ndimage.center_of_mass(tc[blob])} and {np.count_nonzero(tc[blob]) / 10**3} cm^3")


print("\nFor Enhancing Tumor")
for i,blob in enumerate(blob_et):
    print(f"Blob {i}: coordinate {ndimage.center_of_mass(et[blob])} and {np.count_nonzero(et[blob]) / 10**3} cm^3")

For Whole Tumor
Blob 0: coordinate (30.557043041114397, 53.65276439778981, 42.876817371469066) and 180.618 cm^3

For Tumor Core
Blob 0: coordinate (21.879299974221638, 32.3802308595652, 25.608541231060062) and 34.913 cm^3

For Enhancing Tumor
Blob 0: coordinate (23.28733738807231, 32.01900658895084, 27.14660415610745) and 23.676 cm^3


In [38]:
# GLCM

# Based on https://github.com/Prof-Iz/GLCM-from-3D-NumPy-Input

def glcm_3d(input: np.ndarray, delta: tuple[int] = (1, 1, 1), d: int = 1):
    """_summary_

    Args:
        input (np.ndarray): input array. 3D. dtype int
        delta (tuple[int], optional): Direction vector from pixel. Defaults to (1, 1, 1).
        d (int, optional): Distance to check for neighbouring channel. Defaults to 1.

    Raises:
        Exception: if input is not of type dint or is not 3D

    Returns:
        _type_: GLCM Matrix
    """

    if 'int' not in input.dtype.__str__():
        raise Exception("Input should be of dtype Int")

    if len(input.shape) != 3:
        raise Exception("Input should be 3 dimensional")

    offset = (delta[0] * d, delta[1] * d, delta[2] * d)  # offset from each pixel

    x_max, y_max, z_max = input.shape  # boundary conditions during enumeration

    levels = input.max() + 1 # 0:1:n assume contn range of pixel values

    results = np.zeros((levels, levels))  # initialise results error


    for i, v in np.ndenumerate(input):
        x_offset = i[0] + offset[0]
        y_offset = i[1] + offset[1]
        z_offset = i[2] + offset[2]

        if (x_offset >= x_max) or (y_offset >= y_max) or (z_offset >= z_max):
            # if offset out of boundary skip
            continue

        value_at_offset = input[x_offset, y_offset, z_offset]

        results[v, value_at_offset] += 1

    return results / levels**2

In [45]:
# find GLCM for a blob in WT

wt_tumor_boundary_t1    = t1[blob_wt[0]]
wt_tumor_boundary_t1ce  = t1ce[blob_wt[0]]
wt_tumor_boundary_t2    = t2[blob_wt[0]]
wt_tumor_boundary_flair = flair[blob_wt[0]]

wt_tumor_boundary_t1.shape

(66, 110, 79)

In [48]:
glcm = glcm_3d(np.int32(wt_tumor_boundary_t1))

In [90]:
def normalise(P):
    return P / np.sum(P)


def graycoprops(P):

    P = normalise(P)

    energy = np.sum(P**2)

    idx = np.where(P > 0)

    homogeneity = np.sum(P[idx] / (1 + (idx[0] - idx[1]) ** 2))

    contrast = np.sum(P[idx] * (idx[0] - idx[1]) ** 2)

    entropy = np.sum(-P[idx] * np.log(P[idx]))

    # for correlation

    mu = np.sum(idx[0] * P[idx])

    sigma_square = np.sum(P[idx] * (idx[0] - mu) ** 2)

    correlation = np.sum((P[idx] * (idx[0] - mu) * (idx[1] - mu)) / sigma_square)

    return {
        "energy": energy,
        "homogeneity": homogeneity,
        "contrast": contrast,
        "entropy": entropy,
        "correlation": correlation,
    }


In [91]:
print(graycoprops(glcm))

{'energy': 0.010458012361037416, 'homogeneity': 0.14261134493343397, 'contrast': 7161.607885927293, 'entropy': 10.093394917491178, 'correlation': 0.9076729367777118}
